In [34]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Model
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import preprocess_input
import cv2

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

tf.compat.v1.disable_eager_execution()

In [35]:
def load_model_h5(model_file):
    """
    Load the original keras model file
         :param model_file: model file, h5 type
         :return: model
    """
    return load_model(model_file)

def load_img_preprocess(img_path, target_size):
    """
         Load image and preprocess
         :param img_path: image file name
                       target_size: the zoom size of the image to be loaded
                                                 This is a tuple type
         :return: preprocessed image file
    """
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img) # Convert to array form
    img = np.expand_dims(img, axis=0) # Add a one-dimensional batchsize to the picture, directly set it to 1

    img = preprocess_input(img) # standardize the image

    return img

In [36]:
def gradient_compute(model, layername, img):
    """
         Calculate the gradient between the final output of the model and your layer
         And average the gradient of each feature map
         Multiply it with the output of the convolutional layer
         :param model: model
         :param layername: The name of the layer you want to visualize heat
         :param img: preprocessed image
    :return:
         The output value of the convolutional layer multiplied by the average gradient
    """
    preds = model.predict(img)
    idx = np.argmax(preds[0]) # Return the index index of the maximum probability of the predicted picture

    output = model.output[:, idx] # Get the output tensor of our corresponding index
    last_layer = model.get_layer(layername)

    grads = K.gradients(output, last_layer.output)[0]

    pooled_grads = K.mean(grads, axis=(0, 1, 2)) # average each gradient feature map,
                                                # Returns a tensor whose size is the channel dimension
    iterate = K.function([model.input], [pooled_grads, last_layer.output[0]])

    pooled_grads_value, conv_layer_output_value = iterate([img])

    for i in range(pooled_grads.shape[0]):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

    return conv_layer_output_value

In [37]:
def plot_heatmap(conv_layer_output_value, img_in_path, img_out_path):
    """
         Draw a heat map
         :param conv_layer_output_value: Convolutional layer output value
         :param img_in_path: the path of the input image
         :param img_out_path: path of output heat map
    :return:
    """
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)

    img = cv2.imread(img_in_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)

    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimopsed_img = heatmap * 0.4 + img

    cv2.imwrite(img_out_path, superimopsed_img)

In [38]:
img_path = r'./images/0311AD_female.png'
layername = r'conv2d_1'

img = load_img_preprocess(img_path, (224, 224))
model = load_model_h5('MRI_vgg16.h5')

conv_value = gradient_compute(model, layername, img)
plot_heatmap(conv_value, img_path, './packagetest3.jpg')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


/home/trojan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in true_divide
  # This is added back by InteractiveShellApp.init_path()
